In [ ]:
import pandas as pd
from tnseq2.src.analysis import *
from pathlib import Path
import plotnine as p9
import seaborn as sns

In [ ]:
root = "/Users/ansintsova/git_repos/nguyenb_tnseq/data/13_04_results"
counts = 'counts'
results = 'results'
control_file = Path(root)/'controls.txt'

In [ ]:
exp_not_analyzed = ['dnaid1315:TV3371A', 'dnaid1428:TV3652A', 'dnaid1428:TV3522E']

dnaid ='dnaid2023'
d2023=analyze_dnaid(Path(root)/counts, dnaid, control_file, cutoff=0.8, to_filter=0, outdir=Path(root)/results)

dnaid ='dnaid1315'
d1315=analyze_dnaid(Path(root)/counts, dnaid, control_file, cutoff=0.8, to_filter=0, outdir=Path(root)/results)

dnaid ='dnaid1428'
d1428=analyze_dnaid(Path(root)/counts, dnaid, control_file, cutoff=0.8, to_filter=0, outdir=Path(root)/results)

dnaid ='dnaid1429'
d1429=analyze_dnaid(Path(root)/counts, dnaid, control_file, cutoff=0.8, to_filter=0, outdir=Path(root)/results)

In [ ]:
dnaids = ['dnaid1315', 'dnaid1428', 'dnaid1429', 'dnaid2015', 'dnaid2016', 'dnaid2017', 'dnaid2018', 'dnaid2019',
         'dnaid2023', 'dnaid2024', 'dnaid2025', 'dnaid2026', 'dnaid2027', 'dnaid2028', 'dnaid2029' ]
          
        #   'dnaid2030' -> has to be analyzed separately, different design


analyze_dnaid(Path(root)/counts, 'dnaid2019', control_file, cutoff=0.8, to_filter=0, outdir=Path(root)/results)

In [ ]:
for dnaid in dnaids:
    print("")
    print(dnaid)
    print("")
    analyze_dnaid(Path(root)/counts, dnaid, control_file, cutoff=0.8, to_filter=0, outdir=Path(root)/results)

In [ ]:
# Load all the analysis results


In [ ]:
def combine_days(dnaid1, dnaid2, results_dir):
    df1 = pd.read_csv(Path(results_dir)/f'{dnaid1}_final_results.csv').assign(dnaid=f'{dnaid1}:{dnaid2}')
    df2 = pd.read_csv(Path(results_dir)/f'{dnaid2}_final_results.csv').assign(dnaid=f'{dnaid1}:{dnaid2}')
    df = df1.merge(df2, how='outer', on = ['gene', 'locus', 'num_barcodes', 'library', 'barcode', 'sstart', 'sseqid', 'experiment', 'dnaid'])
    df.to_csv(Path(results_dir)/f'{dnaid1}_{dnaid2}_merged_results.csv')
    return df


In [ ]:
to_merge = [('dnaid2015', 'dnaid2016'), ('dnaid2017', 'dnaid2018'), ('dnaid2023', 'dnaid2024'), 
           ('dnaid2025', 'dnaid2026'), ('dnaid2028', 'dnaid2029')]


for d2 in to_merge:
    combine_days(d2[0], d2[1], Path(root)/results)


In [ ]:
singles = ['dnaid1315', 'dnaid1428', 'dnaid1429', 'dnaid2019', 'dnaid2027']



In [ ]:
def load_merged_results(results_dir):
    df = pd.concat([pd.read_csv(file, index_col=0) for file in results_dir.iterdir() if 'merged_results.csv' in file.name])
    return df

def load_singltons(results_dir):
    singles = ['dnaid1315', 'dnaid1428', 'dnaid1429', 'dnaid2019', 'dnaid2027']
    df_list = []
    for s in singles:
        df = pd.read_csv(Path(results_dir)/f'{s}_final_results.csv').assign(dnaid=s)
        df = df.rename({'Unnamed: 0': 'gene'}, axis =1)
        df_list.append(df)
    
    return pd.concat(df_list)
    
    
def get_results(results_dir):
    df = load_merged_results(Path(root)/results)
    df2 = load_singltons(Path(root)/results)
    return pd.concat([df, df2])

fdf = get_results(Path(root)/results)  

In [ ]:
fdf = get_results(Path(root)/results)

#fdf = fdf.dropna(subset=['locus'])

In [ ]:
fdf[['gene', 'locus', 'sstart', 'd1_ci', 'd2_ci', 'd3_ci', 'd4_ci', 'experiment', 'dnaid']].sample(10)

In [ ]:
print(fdf.shape)
print(fdf.gene.nunique())
print(fdf.experiment.nunique())
print(fdf.experiment.unique())

In [ ]:
num_hits = fdf.groupby('gene').experiment.count().reset_index()

In [ ]:
num_hits.experiment.hist(bins=50)

In [ ]:
num_hits_g3 = num_hits[num_hits.experiment > 2]

In [ ]:
num_hits_g3.shape

In [ ]:
1982/2990

In [ ]:
# 44% of genes were only hit once or twice



In [ ]:
# Day 1
# Genes that were significant in at least 1 library
# of libraries this gene was found in
# % of libraries it was a hit in
d1_sig = fdf[fdf.d1_padj < 0.05] # 342 genes were signifcantly up/down at least ones
d1_genes = d1_sig.gene.unique()

In [ ]:
len(d1_genes)

In [ ]:
d1_all = fdf[fdf.gene.isin(d1_genes)]

In [ ]:
d1_cons = d1_all.dropna(subset=['experiment']).groupby('gene').agg({'experiment':['count'], 'd1_padj':['min', 'max', num_sig]}).reset_index()
d1_cons.columns = ['gene', 'exp_count', 'padj_min', 'padj_max', 'numSig']

In [ ]:
d1_cons.sort_values("numSig", ascending=False)

In [ ]:
def num_sig(x):
    return sum([i < 0.05 for i in x])/len(x)

In [ ]:
d1_cons.numSig.hist(bins=50)

In [ ]:
d1_cons.plot("exp_count", "numSig", kind='scatter', alpha=0.5)

In [ ]:
d1_cons[d1_cons.numSig >0.4]

In [ ]:
d1_genes

In [ ]:
d1_cons[d1_cons.gene == 'yiiG']

In [ ]:
fdf[fdf.gene == 'rfaI'][['gene','d3_num_samples', 'd3_fitness_mean', 'd3_ci', 'd3_zscore', 'd3_padj', 'experiment', 'dnaid']].dropna()

In [ ]:
# WT 

def cntrl_df(fdf, phenotype = 'wt'):
    wt = fdf[fdf.gene.isin([i for i in fdf.gene if phenotype in str(i)])]
    wt = wt[['gene', 'experiment', 'dnaid', 'd1_fitness_mean', 'd2_fitness_mean', 'd3_fitness_mean', 'd4_fitness_mean']].drop_duplicates()
    wt = wt.melt(id_vars=['gene', 'experiment', 'dnaid'],var_name = 'day', value_name='fitness')

    replace_map = {'d1_fitness_mean': 'Day 1', 
                      'd2_fitness_mean': 'Day 2',
                      'd3_fitness_mean': 'Day 3',
                      'd4_fitness_mean': 'Day 4'}
    wt = wt.replace({'day':replace_map})
    return wt

In [ ]:

wt = cntrl_df(fdf)
p9.options.figure_size = (20, 25)
p = p9.ggplot(p9.aes(x='day', y='fitness', color='dnaid'), wt)
(p + p9.geom_jitter(size=5, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("WT Fitness\n")
+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
+ p9.facet_wrap('~ experiment', ncol=4))

In [ ]:
hyb = cntrl_df(fdf, 'hyb-')
hyb

In [ ]:
hyb = cntrl_df(fdf, 'hyb-')
p9.options.figure_size = (20, 25)
p = p9.ggplot(p9.aes(x='day', y='fitness', color='dnaid'), hyb)
(p + p9.geom_jitter(size=5, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("hyb Fitness\n")
+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
+ p9.facet_wrap('~ experiment', ncol=4))

In [ ]:
chey = cntrl_df(fdf, 'chey-')
chey

In [ ]:
chey = cntrl_df(fdf, 'chey-')
p9.options.figure_size = (20, 25)
p = p9.ggplot(p9.aes(x='day', y='fitness', color='dnaid'), chey)
(p + p9.geom_jitter(size=5, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("chey Fitness\n")
+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
+ p9.facet_wrap('~ experiment', ncol=4))

In [ ]:
ssa = cntrl_df(fdf, 'ssaV_')
ssa

In [ ]:

p9.options.figure_size = (20, 25)
p = p9.ggplot(p9.aes(x='day', y='fitness', color='dnaid'), ssa)
(p + p9.geom_jitter(size=5, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("ssaV Fitness\n")
+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
+ p9.facet_wrap('~ experiment', ncol=4))

In [ ]:
# Genes of Interest

In [ ]:
def get_gene(fdf, search_str='rfa'):
    lps = fdf[fdf.gene.isin([i for i in fdf.gene if search_str in i])]
    lps = lps[['gene', 'experiment', 'dnaid', 'd1_ci', 'd2_ci', 'd3_ci', 'd4_ci']].drop_duplicates()
    lps = lps.melt(id_vars=['gene', 'experiment', 'dnaid'],var_name = 'day', value_name='CI')

    replace_map_ci = {'d1_ci': 'Day 1', 
                          'd2_ci': 'Day 2',
                          'd3_ci': 'Day 3',
                          'd4_ci': 'Day 4'}
    lps = lps.replace({'day':replace_map_ci})
    return lps

In [ ]:
rfa = get_gene(fdf, search_str='rfa')
rfb = get_gene(fdf, search_str='rfb')
frd = get_gene(fdf, search_str='frd')
dcu = get_gene(fdf, search_str='dcu')
hil = get_gene(fdf, search_str='hil')

In [ ]:
p9.options.figure_size = (15, 25)
p = p9.ggplot(p9.aes(x='day', y='CI', color='experiment'), rfa)
(p + p9.geom_jitter(size=3, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("rfa genes\n")
#+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
 + p9.scale_y_log10()
+ p9.facet_wrap('~ gene', ncol=4))

In [ ]:
p9.options.figure_size = (15, 25)
p = p9.ggplot(p9.aes(x='day', y='CI', color='experiment'), rfb)
(p + p9.geom_jitter(size=3, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("rfb genes\n")
#+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
 + p9.scale_y_log10()
+ p9.facet_wrap('~ gene', ncol=4))

In [ ]:
p9.options.figure_size = (10, 10)
p = p9.ggplot(p9.aes(x='day', y='CI', color='experiment'), frd)
(p + p9.geom_jitter(size=3, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("frd genes\n")
#+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
 + p9.scale_y_log10()
+ p9.facet_wrap('~ gene', ncol=4))

In [ ]:
p9.options.figure_size = (10, 5)
p = p9.ggplot(p9.aes(x='day', y='CI', color='experiment'), hil)
(p + p9.geom_jitter(size=3, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("hil genes\n")
#+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
 + p9.scale_y_log10()
+ p9.facet_wrap('~ gene', ncol=4))

In [ ]:
p9.options.figure_size = (12, 10)
p = p9.ggplot(p9.aes(x='day', y='CI', color='experiment'), dcu)
(p + p9.geom_jitter(size=3, height=0, width=0.05) 
+ p9.theme_classic()
+ p9.ggtitle("dcu genes\n")
#+ p9.labs( y = 'Fitness (log2FC)')
+ p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
#+ p9.scale_y_continuous(breaks=[-1.5, -1, -0.5, 0, 0.5, 1, 1.5], limits=[-1.5, 1.5])
 + p9.scale_y_log10()
+ p9.facet_wrap('~ gene', ncol=3))

In [ ]:
# Test
genes_only = fdf.dropna(subset=['locus'])
d1_hits = genes_only[genes_only.d1_padj < 0.01].gene.values

In [ ]:
cnt_df = load_files(dnaids, Path(root)/counts)

In [ ]:
cnt_df.sample(10)

In [ ]:
def summarize_dpi(cnts_df, results_df, dpi='d1'):
    # Get the hits
    genes_only = results_df.dropna(subset=['locus'])
    hits = genes_only[genes_only[f'{dpi}_padj'] < 0.01].gene.values
    # Get the counts
    cnts = cnts_df[cnts_df.ShortName.isin(hits)]
    cnts_sum = cnts.groupby(["ShortName", 'day']).agg({'cnt':['median']}).reset_index()
    cnts_sum.columns = ['gene', 'day', 'cnt_median']
    cnts_sum = cnts_sum[cnts_sum.day.isin(['d0', dpi])]
    cnts_final = cnts_sum[['gene', 'day', 'cnt_median']]
    cnts_final = cnts_final.pivot(index='gene', columns='day').reset_index()
    cnts_final.columns = ['gene', 'inoculum', 'cnt']
    
    #Get num of experiments
    hit_stat = (results_df[results_df['gene'].isin(hits)].groupby('gene')
                   .agg({'experiment':['nunique'], f'{dpi}_padj':[len_sig]}).reset_index())
    hit_stat.columns = ['gene', 'num_exp', 'num_sig']
    tograph = hit_stat.merge(cnts_final, on='gene')
    tograph['phits'] = tograph['num_sig']/tograph['num_exp']
    return tograph

def len_sig(x):
    x = x.drop_duplicates()
    return sum([i < 0.05 for i in x])

In [ ]:
d1_cnts = cnt_df[cnt_df.ShortName.isin(d1_hits)]

In [ ]:
d1_cnts.sample(10)

In [ ]:
d1_cnts_sum = d1_cnts.groupby(["ShortName", 'day']).agg({'cnt':['median', 'count', 'max', 'min']}).reset_index()
d1_cnts_sum.columns = ['gene', 'day', 'cnt_median', 'cnt_count', 'cnt_max', 'cnt_min']
d1_cnts_sum = d1_cnts_sum[d1_cnts_sum.day.isin(['d0', 'd1'])]

d1_cnts_final = d1_cnts_sum[['gene', 'day', 'cnt_median']]
d1_cnts_final = d1_cnts_final.pivot(index='gene', columns='day').reset_index()
d1_cnts_final.columns = ['gene', 'inoculum', 'day1_cnt']

In [ ]:
d1_hit_stat = d1_sum.groupby('gene').agg({'experiment':['nunique'], 'd1_padj':[len_sig]}).reset_index()
d1_hit_stat.columns = ['gene', 'num_exp', 'num_sig']
d1_hit_stat
d1_tograph = d1_hit_stat.merge(d1_cnts_final, on='gene')
d1_tograph['phits'] = d1_tograph['num_sig']/d1_tograph['num_exp']

In [ ]:
d1_tograph

In [ ]:
import matplotlib
matplotlib.cm.cmap_d.keys()

In [ ]:
dpi1 = summarize_dpi(cnt_df, fdf, dpi='d1')

In [ ]:
fdf[fdf.gene == 'nagC'][['gene', 'experiment', 'd2_padj']].drop_duplicates()

In [ ]:
p9.options.figure_size = (10, 10)
p = p9.ggplot(p9.aes(x='inoculum', y='cnt'), dpi1)
(p + p9.geom_point(p9.aes(fill='phits', size='num_exp'))
+p9.scale_y_log10()
 +p9.scale_x_log10()
 +p9.geom_text(p9.aes(x='inoculum', y='cnt', label='gene'),
                  color="black",nudge_y=0.08,
                  data=dpi1[dpi1['phits'] >0.5])
 +p9.theme_classic()
 + p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
 +p9.ggtitle("1 DPI")
 +p9.labs(y ="1 DPI, Median Reads", x= "Inoculum, Median Reads")
 +p9.geom_abline(intercept=0, slope=1)
 
 +p9.scale_fill_continuous('hot_r')
)


In [ ]:
dpi2 = summarize_dpi(cnt_df, fdf, dpi='d2')

In [ ]:
dpi2 = summarize_dpi(cnt_df, fdf, dpi='d2')
p9.options.figure_size = (10, 10)
p = p9.ggplot(p9.aes(x='inoculum', y='cnt'), dpi2)
(p + p9.geom_point(p9.aes(fill='phits', size='num_exp'))
+p9.scale_y_log10()
 +p9.scale_x_log10()
 +p9.geom_text(p9.aes(x='inoculum', y='cnt', label='gene'),
                  color="black",nudge_y=0.08,
                  data=dpi2[dpi2['phits'] >0.5])
 +p9.theme_classic()
 + p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
 +p9.ggtitle("2 DPI")
 +p9.labs(y ="2 DPI, Median Reads", x= "Inoculum, Median Reads")
 +p9.geom_abline(intercept=0, slope=1)
 
 +p9.scale_fill_continuous('hot_r')
)


In [ ]:
dpi3 = summarize_dpi(cnt_df, fdf, dpi='d3')
dpi4 = summarize_dpi(cnt_df, fdf, dpi='d4')

In [ ]:
dpi3 = summarize_dpi(cnt_df, fdf, dpi='d3')
p9.options.figure_size = (10, 10)
p = p9.ggplot(p9.aes(x='inoculum', y='cnt'), dpi3)
(p + p9.geom_point(p9.aes(fill='phits', size='num_exp'))
+p9.scale_y_log10()
 +p9.scale_x_log10()
 +p9.geom_text(p9.aes(x='inoculum', y='cnt', label='gene'),
                  color="black",nudge_y=0.08,
                  data=dpi3[dpi3['phits'] >0.5])
 +p9.theme_classic()
 + p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
 +p9.ggtitle("3 DPI")
 +p9.labs(y ="3 DPI, Median Reads", x= "Inoculum, Median Reads")
 +p9.geom_abline(intercept=0, slope=1)
 
 +p9.scale_fill_continuous('hot_r')
)


In [ ]:
dpi4 = summarize_dpi(cnt_df, fdf, dpi='d4')
p9.options.figure_size = (10, 10)
p = p9.ggplot(p9.aes(x='inoculum', y='cnt'), dpi4)
(p + p9.geom_point(p9.aes(fill='phits', size='num_exp'))
+p9.scale_y_log10()
 +p9.scale_x_log10()
 +p9.geom_text(p9.aes(x='inoculum', y='cnt', label='gene'),
                  color="black",nudge_y=0.08,
                  data=dpi4[dpi4['phits'] >0.5])
 +p9.theme_classic()
 + p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 24, hjust = 0), # Title size and font.
    axis_text = p9.element_text(family = "Arial", size = 18), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 18), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1.5), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_text = p9.element_text(family = "Arial", size = 18),
    strip_text = p9.element_text(family = "Arial", size = 18),
  )
 +p9.ggtitle("4 DPI")
 +p9.labs(y ="4 DPI, Median Reads", x= "Inoculum, Median Reads")
 +p9.geom_abline(intercept=0, slope=1)
 
 +p9.scale_fill_continuous('hot_r')
)

In [ ]:
def num_sig(x):
    return sum([i < 0.05 for i in x])/len(x)

In [ ]:
def lenx(x):
    return len(x.dropna())

In [ ]:
fdf.shape

In [ ]:
fdf[['gene', 'locus', 'library', 'experiment', 'dnaid']].head(25)

In [ ]:
fdf.to_csv(Path(root)/results/"16_04_final_results_merged.csv", index=False)